In [4]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
#### importing packages

import pandas as pd
import os
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By   ### para emplear los .find_element(By.)
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup as bs

from selenium.webdriver.chrome.service import Service   ### para instalar el drive
from selenium.webdriver.support.ui import WebDriverWait    ### para las alertas
from selenium.common.exceptions import TimeoutException    ### para las alertas
from selenium.webdriver.support import expected_conditions as EC  ### para las alertas
import re

In [16]:
##### importing the web to chromedriver  
os.chdir("..\\output")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#driver = webdriver.Chrome(ChromeDriverManager().install())
enlaces = "https://plc.mintransporte.gov.co/Runtime/empresa/ctl/SiceTAC/mid/417"
driver.get(enlaces)
driver.maximize_window()
body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML')
time.sleep(1)

In [83]:
#periodo_box= driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$periodoJ']")
#periodo_box.click()
periodo_year= driver.find_elements(By.XPATH,"//select[@class='ui-datepicker-year']/option")
periodo_year = [d.text for d in periodo_year]


In [84]:
### periodos disponibles 
## para hcer el loop por periodo 
year=[]
month=[]

for y in periodo_year: 
    selectyear= Select(driver.find_element(By.XPATH,"//select[@class='ui-datepicker-year']"))
    selectyear.select_by_visible_text(y)
    periodo_month= driver.find_elements(By.XPATH,"//select[@class='ui-datepicker-month']/option")
    periodo_month = [d.text for d in periodo_month]
    for m in periodo_month:
        year.append(y)
        month.append(m)

In [49]:
periodos = pd.DataFrame()
periodos["year"] = year
periodos["month"] = month

In [51]:
periodos.to_excel("periodos.xlsx", index=False)

In [ ]:
#####  Código para obtener las conbinaciones de configuración, tipo de carga y tipo de unidad permitidas. 

config = driver.find_elements(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$CONFIGURACION']/option")
config = [d.text for d in config]
config.pop(0) 
carga = driver.find_elements(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$TIPOCARGA']/option")
carga = [d.text for d in carga]
carga.pop(0) 
tipo = driver.find_elements(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$UNIDADTRANSPORTE']/option")
tipo = [d.text for d in tipo]
tipo.pop(0)
origen = driver.find_elements(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$ORIGEN']/option")
origen = [d.text for d in origen]
origen.pop(0) 
destino = driver.find_elements(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$DESTINO']/option")
destino = [d.text for d in destino]
destino.pop(0)

 
h_cargue = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$HORASCARGUE']")
h_cargue.clear()
h_cargue.send_keys("1")
h_descargue = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$HORASDESCARGUE']") 
h_descargue.clear()
h_descargue.send_keys("1")
h_e_cargue = driver.find_element(By.XPATH,"//input[@name = 'dnn$ctr417$SiceTAC$HORASESPERACARGUE']")
h_e_cargue.clear()
h_e_cargue.send_keys("1")
h_e_descargue = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$HORASESPERADESCARGUE']")
h_e_descargue.clear()
h_e_descargue.send_keys("1")
calculo_text = driver.find_element(By.XPATH,"//span[@id='dnn_ctr417_SiceTAC_Cat']").text
calculo_text=calculo_text.split()
calculo_text= int(calculo_text[2])+int(calculo_text[4])
calculo = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$Resultado']")
calculo.send_keys(calculo_text)
calcular = driver.find_element(By.XPATH,"//a[@id='dnn_ctr417_SiceTAC_btCalcular']")
calcular.click()
alert = driver.switch_to.alert
alert.text
alert.accept()

# #Combinaciones posibles
#c_oper = driver.find_element(By.XPATH,"//td[@colspan='6']").text 
#c_oper == "No hay conceptos de costo para mostrar"

config_v = []
tipo_carga = []
tipo_unidad = []
for co in config:
    for ca in carga:
        for ti in tipo:
            calculo = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$Resultado']")
            calculo.clear()
            config_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$CONFIGURACION']"))
            config_op.select_by_visible_text(co)
            carga_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$TIPOCARGA']"))
            carga_op.select_by_visible_text(ca)
            tipo_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$UNIDADTRANSPORTE']"))
            tipo_op.select_by_visible_text(ti)
            time.sleep(2)
            org = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$ORIGEN']"))
            org.select_by_visible_text(origen[17])
            org = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$DESTINO']"))
            org.select_by_visible_text(destino[8])
            calculo_text = driver.find_element(By.XPATH,"//span[@id='dnn_ctr417_SiceTAC_Cat']").text
            calculo_text=calculo_text.split()
            calculo_text= int(calculo_text[2])+int(calculo_text[4])
            calculo = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$Resultado']")
            calculo.send_keys(calculo_text)
            calcular = driver.find_element(By.XPATH,"//a[@id='dnn_ctr417_SiceTAC_btCalcular']")
            time.sleep(2)
            calcular.click()
            time.sleep(2)
            try:    
                WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                   'Timed out waiting for PA creation ' +
                                   'confirmation popup to appear.')
                alert = driver.switch_to.alert
                alert.accept()
                print("alert accepted")
            except TimeoutException:
                print("no alert")
            try:
                c_oper = driver.find_element(By.XPATH,"//td[@colspan='6']").text 
            except:
                c_oper = ""
            if c_oper != "No hay conceptos de costo para mostrar":
               config_v.append(co) 
               tipo_carga.append(ca)
               tipo_unidad.append(ti)

               
df = pd.DataFrame()
df["config_v"] = config_v
df["tipo_carga"] = tipo_carga
df["tipo_unidad"] = tipo_unidad
#guardar info

df.to_excel("combinaciones.xlsx", index=False)

In [3]:
periodos= pd.read_excel("periodos.xlsx")
OD = pd.read_excel("OD.xlsx")
combinaciones= pd.read_excel('combinaciones.xlsx')

In [4]:
### Panel costos operativos resumen
cost_panel_resumen = pd.DataFrame(columns = ['origen', 
'destino',
'config_v',
'tipo_carga',
'tipo_unidad',
'month',
'year', 
'Tonelada x KM Movilzación',
 'Costo Tonelada Movilización',
 'Costo Movilización Carga',
 'Costo Hora Adicional',
 'Horas de Espera',
 'Costo Tiempos de Espera',
 'Tonelada x KM del Viaje',
 'Costo Tonelada del Viaje',
 'Costo Total del Viaje',
 'Costo x KM Movilización',
 'Costo x KM del Viaje'])



In [19]:
it=0

for x in range(len(combinaciones)):
    for O in range(len(OD)): 
        for p in  range(len(periodos)):                 
                    config_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$CONFIGURACION']"))
                    config_op.select_by_visible_text(combinaciones.loc[x,"config_v"])
                    carga_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$TIPOCARGA']"))
                    carga_op.select_by_visible_text(combinaciones.loc[x,"tipo_carga"])
                    tipo_op = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$UNIDADTRANSPORTE']"))
                    tipo_op.select_by_visible_text(combinaciones.loc[x,"tipo_unidad"])
                    time.sleep(1.5)
                    org = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$ORIGEN']"))
                    org.select_by_visible_text(OD.loc[O,"origen"])
                    time.sleep(1.5)
                    des = Select(driver.find_element(By.XPATH,"//select[@name='dnn$ctr417$SiceTAC$DESTINO']"))
                    des.select_by_visible_text(OD.loc[O,"destino"])
                    time.sleep(2)
                    
                    ### Seleccionar periodo
                    periodo_box= driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$periodoJ']")
                    periodo_box.click()
                    
                    WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//select[@class='ui-datepicker-year']"))
                    )
                    selectyear= Select(driver.find_element(By.XPATH,"//select[@class='ui-datepicker-year']"))
                    selectyear.select_by_visible_text(str(periodos.loc[p,"year"]))
                    time.sleep(1.5)
                    selectmonth= Select(driver.find_element(By.XPATH,"//div[@class='ui-datepicker-header ui-widget-header ui-helper-clearfix ui-corner-all']//select[@class='ui-datepicker-month']"))
                    selectmonth.select_by_visible_text(periodos.loc[p,"month"])

                    periodo_close= driver.find_element(By.XPATH,"//button[@class='ui-datepicker-close ui-state-default ui-priority-primary ui-corner-all']")
                    periodo_close.click()
                    
                    time.sleep(1.5)
                    
                    

                    #### calculo 

                    calculo_text = driver.find_element(By.XPATH,"//span[@id='dnn_ctr417_SiceTAC_Cat']").text
                    calculo_text=calculo_text.split()
                    calculo_text= int(calculo_text[2])+int(calculo_text[4])
                    calculo = driver.find_element(By.XPATH,"//input[@name='dnn$ctr417$SiceTAC$Resultado']")
                    calculo.send_keys(calculo_text)
                    
                    WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,"//a[@id='dnn_ctr417_SiceTAC_btCalcular']")))

                    calcular = driver.find_element(By.XPATH,"//a[@id='dnn_ctr417_SiceTAC_btCalcular']")
                    calcular.click()
                    time.sleep(2)

                    alert = driver.switch_to.alert
                    alert.accept()
                    print("alert accepted")
                    

                    for col in periodos.columns: 
                           cost_panel_resumen.loc[it, col]= periodos.loc[it,col]
                    for col in combinaciones.columns: 
                           cost_panel_resumen.loc[it, col]= combinaciones.loc[it,col]
                    for col in OD.columns: 
                           cost_panel_resumen.loc[it, col]= OD.loc[it,col]
                     
                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOMOVKM']")
                    cost_panel_resumen.loc[it, 'Tonelada x KM Movilzación' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOTONMOV']")
                    cost_panel_resumen.loc[it, 'Costo Tonelada Movilización' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOMOVCARGA']")
                    cost_panel_resumen.loc[it, 'Costo Movilización Carga' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOHORAADL']")
                    cost_panel_resumen.loc[it, 'Costo Hora Adicional' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$HORASESPERA']")
                    cost_panel_resumen.loc[it, 'Horas de Espera' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOTIEMPOESPERA']")
                    cost_panel_resumen.loc[it, 'Costo Tiempos de Espera' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOTONELADAKMTOTAL']")
                    cost_panel_resumen.loc[it, 'Tonelada x KM del Viaje' ]=  cost_movkm.get_attribute("value")                    

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOTONELADATOTAL']")
                    cost_panel_resumen.loc[it, 'Costo Tonelada del Viaje' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOTOTALVIAJE']")
                    cost_panel_resumen.loc[it, 'Costo Total del Viaje' ]=  cost_movkm.get_attribute("value")

                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOMOVILKM']")
                    cost_panel_resumen.loc[it, 'Costo x KM Movilización' ]=  cost_movkm.get_attribute("value")
                    
                    cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOVIAJEKM']")
                    cost_panel_resumen.loc[it, 'Costo x KM del Viaje' ]=  cost_movkm.get_attribute("value")
                    
                    it= it+1
                    #driver.execute_script("window.history.go(-1)")
                    driver.get(enlaces)
                    time.sleep(2)

alert accepted


ElementNotInteractableException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=111.0.5563.111)
Stacktrace:
Backtrace:
	(No symbol) [0x004EDCE3]
	(No symbol) [0x004839D1]
	(No symbol) [0x00394DA8]
	(No symbol) [0x003976FB]
	(No symbol) [0x003975D0]
	(No symbol) [0x00397850]
	(No symbol) [0x003C3173]
	(No symbol) [0x003B90D4]
	(No symbol) [0x003DAECC]
	(No symbol) [0x003B8726]
	(No symbol) [0x003DB224]
	(No symbol) [0x003ED57C]
	(No symbol) [0x003DACC6]
	(No symbol) [0x003B6F68]
	(No symbol) [0x003B80CD]
	GetHandleVerifier [0x00763832+2506274]
	GetHandleVerifier [0x00799794+2727300]
	GetHandleVerifier [0x0079E36C+2746716]
	GetHandleVerifier [0x00596690+617600]
	(No symbol) [0x0048C712]
	(No symbol) [0x00491FF8]
	(No symbol) [0x004920DB]
	(No symbol) [0x0049C63B]
	BaseThreadInitThunk [0x76AC00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77C67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77C67B8E+238]


In [20]:
cost_panel_resumen

,origen,destino,config_v,tipo_carga,tipo_unidad,month,year,Tonelada x KM Movilzación,Costo Tonelada Movilización,Costo Movilización Carga,Costo Hora Adicional,Horas de Espera,Costo Tiempos de Espera,Tonelada x KM del Viaje,Costo Tonelada del Viaje,Costo Total del Viaje,Costo x KM Movilización,Costo x KM del Viaje
0,AGUACHICA-CESAR,BARRANCABERMEJA,Camión dos ejes - Sencillo,General,ESTACAS,Jul,2021,$283.17,"$53,010.26","$477,092.25","$24,240",17.2,"$416,933.35",$530.64,"$99,336.18","$894,025.60","$2,548.57","$4,775.78"


In [66]:
len(periodos)
for p in   range(len(periodos)):
    print(periodos.loc[p,"year"])

2021
2021
2021
2021
2021
2021
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2023
2023
2023


In [37]:
cost_movkm= driver.find_element(By.XPATH, "//input[@name='dnn$ctr417$SiceTAC$COSTOMOVKM']")
cost_movkm.get_attribute("value")

'$288.85'

In [41]:
cost_movkm= driver.find_elements(By.XPATH, "//div[@class='panel-body']/input")
cost_movkm= [d.get_attribute("value") for d in cost_movkm ]
cost_movkm

[]